# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [78]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [79]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [80]:
def build_classifiers():
    classifiers = [
        LinearRegression(),
        KNeighborsClassifier(),
        SVC(),
        DecisionTreeClassifier(),
        GaussianNB(),
        QuadraticDiscriminantAnalysis()
    ]
    return classifiers

In [81]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        classifier.fit(data_set, labels)
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [82]:
classifiers = build_classifiers()
accuracies = []
seen_permutations = set()

from itertools import permutations
from copy import deepcopy
import pandas as pd

for selected_classifiers in permutations(classifiers, 3):
    three_classifiers = selected_classifiers[:3]

    unique_permutation = tuple(sorted([str(classifier) for classifier in three_classifiers]))
    if unique_permutation not in seen_permutations:
        seen_permutations.add(unique_permutation)
        predicted = build_stacked_classifier(three_classifiers)
        accuracy = accuracy_score(test_labels, predicted)
        accuracies.append((accuracy, str(type(three_classifiers[0]).__name__), str(type(three_classifiers[1]).__name__), str(type(three_classifiers[2]).__name__)))

sorted_accuracies = sorted(accuracies, key=lambda x: x[0], reverse=True)
df = pd.DataFrame(sorted_accuracies,
                  columns=['Accuracy', 'Classifier 1', 'Classifier 2', 'Classifier 3'])

# Display the top 10 results
print(df.head(10))

   Accuracy            Classifier 1            Classifier 2  \
0      0.90    KNeighborsClassifier  DecisionTreeClassifier   
1      0.85    KNeighborsClassifier                     SVC   
2      0.85    KNeighborsClassifier                     SVC   
3      0.85    KNeighborsClassifier                     SVC   
4      0.85    KNeighborsClassifier              GaussianNB   
5      0.85                     SVC  DecisionTreeClassifier   
6      0.85  DecisionTreeClassifier              GaussianNB   
7      0.75        LinearRegression    KNeighborsClassifier   
8      0.75        LinearRegression  DecisionTreeClassifier   
9      0.75                     SVC              GaussianNB   

                    Classifier 3  
0  QuadraticDiscriminantAnalysis  
1         DecisionTreeClassifier  
2                     GaussianNB  
3  QuadraticDiscriminantAnalysis  
4  QuadraticDiscriminantAnalysis  
5  QuadraticDiscriminantAnalysis  
6  QuadraticDiscriminantAnalysis  
7  QuadraticDiscriminantAn

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [84]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [102]:
def set_new_weights(model):
    predicted = model.predict(test_set)
    weights[:] = (1 + (predicted != test_labels)) / np.sum(1 + (predicted != test_labels))
    return weights

Train the classifier with the code below:

In [103]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00131234 0.00065617 0.00065617 0.00131234 0.00065617 0.00131234
 0.00065617 0.00065617 0.00131234 0.00065617 0.00065617 0.00131234
 0.00131234 0.00131234 0.00131234 0.00065617 0.00131234 0.00065617
 0.00065617 0.00131234 0.00131234 0.00065617 0.00065617 0.00065617
 0.00065617 0.00065617 0.00065617 0.00065617 0.00131234 0.00131234
 0.00065617 0.00131234 0.00131234 0.00131234 0.00065617 0.00131234
 0.00131234 0.00065617 0.00131234 0.00065617 0.00131234 0.00131234
 0.00065617 0.00131234 0.00065617 0.00131234 0.00131234 0.00065617
 0.00065617 0.00131234 0.00065617 0.00065617 0.00065617 0.00065617
 0.00065617 0.00131234 0.00065617 0.00131234 0.00131234 0.00131234
 0.00131234 0.00065617 0.00065617 0.00065617 0.00065617 0.00065617
 0.00065617 0.00131234 0.00131234 0.00131234 0.00131234 0.00065617
 0.00131234 0.00065617 0.00131234 0.00131234 0.00131234 0.00131234
 0.00131234 0.00131234 0.00131234 0.00065617 0.00065617 0.00065617
 0.00131234 0.00131234 0.00131234 0.00131234 0.00065617 0.0013

Set the validation data set:

In [104]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [100]:
def get_prediction(x):
    votes = np.zeros(len(classifiers))
    for idx, model in enumerate(classifiers):
        votes[idx] = model.predict(x.reshape(1, -1))[0]
    return np.sign(np.sum(votes))

Test it:

In [101]:
prediction = get_prediction(validate_x)[0]

print(prediction)

IndexError: invalid index to scalar variable.